In [115]:
import pandas as pd

In [116]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
import re

In [118]:
df=pd.read_csv('Restaurant_Reviews 1.tsv',delimiter='\t',quoting=3)

In [119]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [120]:
df.shape

(1000, 2)

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [122]:
import string

corpus=[]
for i in range(0,1000):
  reveiw=re.sub(pattern='[^a-zA-Z]',repl=' ',string=df['Review'][i])
  reveiw=reveiw.lower()
  reveiw_word=reveiw.split()
  reveiw_word=[word for word in reveiw_word if not word in set(stopwords.words('english'))]
  ps=PorterStemmer()
  reveiw=[ps.stem(word) for word in reveiw_word]
  reveiw=' '.join(reveiw)
  corpus.append(reveiw)

In [123]:
corpus[:10]

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch']

In [124]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


tfidf_df= pd.DataFrame(data=X.todense(), columns=vectorizer.get_feature_names_out())
print("TF-IDF Matrix:")
print(tfidf_df.tail(30))

TF-IDF Matrix:
     absolut  absolutley  accid  accommod  accomod  accordingli  account  ach  \
970      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
971      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
972      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
973      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
974      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
975      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
976      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
977      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
978      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
979      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
980      0.0         0.0    0.0       0.0      0.0          0.0      0.0  0.0   
981      0.0 

In [125]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [126]:
scaler = StandardScaler()
data_standardized = scaler.fit_transform(tfidf_df)

In [127]:
# Step 2: Apply PCA
pca = PCA(n_components=2)  # Reduce to 2 dimensions
principal_components = pca.fit_transform(data_standardized)

In [128]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [129]:
df.columns

Index(['Review', 'Liked'], dtype='object')

In [130]:
y = df['Liked']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [131]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()

In [132]:
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

In [133]:
from sklearn.metrics import accuracy_score, classification_report

In [134]:
# Fit GaussianNB (requires dense data)
clf1.fit(X_train_dense, y_train)
y_pred_gnb = clf1.predict(X_test_dense)
print("\nGaussianNB Accuracy:", accuracy_score(y_test, y_pred_gnb))
print("\nGaussianNB Classification Report:\n", classification_report(y_test, y_pred_gnb))


GaussianNB Accuracy: 0.69

GaussianNB Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.56      0.64        96
           1       0.67      0.81      0.73       104

    accuracy                           0.69       200
   macro avg       0.70      0.69      0.68       200
weighted avg       0.70      0.69      0.68       200



In [135]:
# Fit MultinomialNB (works with sparse data)
clf2.fit(X_train, y_train)
y_pred_mnb = clf2.predict(X_test)
print("\nMultinomialNB Accuracy:", accuracy_score(y_test, y_pred_mnb))
print("\nMultinomialNB Classification Report:\n", classification_report(y_test, y_pred_mnb))


MultinomialNB Accuracy: 0.745

MultinomialNB Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.76      0.74        96
           1       0.77      0.73      0.75       104

    accuracy                           0.74       200
   macro avg       0.75      0.75      0.74       200
weighted avg       0.75      0.74      0.75       200



In [136]:
# Fit BernoulliNB (works with sparse data)
clf3.fit(X_train, y_train)
y_pred_bnb = clf3.predict(X_test)
print("\nBernoulliNB Accuracy:", accuracy_score(y_test, y_pred_bnb))
print("\nBernoulliNB Classification Report:\n", classification_report(y_test, y_pred_bnb))


BernoulliNB Accuracy: 0.745

BernoulliNB Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.74      0.74        96
           1       0.76      0.75      0.75       104

    accuracy                           0.74       200
   macro avg       0.74      0.74      0.74       200
weighted avg       0.75      0.74      0.75       200



In [137]:
from sklearn.linear_model import LogisticRegression
clf4 = LogisticRegression(max_iter=1000)

# Fit Logistic Regression
clf4.fit(X_train, y_train)
y_pred_lr = clf4.predict(X_test)
print("\nLogistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nLogistic Regression Classification Report:\n", classification_report(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.74

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.85      0.76        96
           1       0.82      0.63      0.72       104

    accuracy                           0.74       200
   macro avg       0.75      0.74      0.74       200
weighted avg       0.76      0.74      0.74       200



In [138]:
from sklearn.ensemble import RandomForestClassifier
clf5 = RandomForestClassifier(n_estimators=100)

# Fit Random Forest Classifier
clf5.fit(X_train, y_train)
y_pred_rf = clf5.predict(X_test)
print("\nRandom Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nRandom Forest Classification Report:\n", classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.7

Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.88      0.74        96
           1       0.82      0.54      0.65       104

    accuracy                           0.70       200
   macro avg       0.73      0.71      0.69       200
weighted avg       0.73      0.70      0.69       200



In [139]:
from sklearn.svm import SVC
clf6 = SVC()

# Fit Support Vector Machine
clf6.fit(X_train, y_train)
y_pred_svc = clf6.predict(X_test)
print("\nSVM Accuracy:", accuracy_score(y_test, y_pred_svc))
print("\nSVM Classification Report:\n", classification_report(y_test, y_pred_svc))


SVM Accuracy: 0.74

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.86      0.76        96
           1       0.83      0.62      0.71       104

    accuracy                           0.74       200
   macro avg       0.76      0.74      0.74       200
weighted avg       0.76      0.74      0.74       200

